In [1]:
from pymongo import MongoClient


client = MongoClient("localhost", 27017)

# Assign the database to a variable
db = client.NonRelProj 

# Take the collection "Reviews"
reviews = db.Reviews

# Take the collection "MetaData"
meta = db.MetaData

Another important element that people might want to understand is whether the restaurant has electronic payment options. Hence, it is first necessary to know which are all the distict payment options among all restaurant and then filter by the electronic ones.

In [2]:
# Aggregate to unwind the 'Payments' 
# field and get distinct values
distinct_payments = meta.aggregate([
    # Unwind the Payments array
    {"$unwind": "$MISC.Payments"},  
    # Group by payment method
    {"$group": {"_id": "$MISC.Payments"}},  
    # Sort alphabetically
    {"$sort": {"_id": 1}}  
])

# Print the distinct payment methods
print("Distinct Payment Methods:")
for payment in distinct_payments:
    print(payment["_id"])


Distinct Payment Methods:
Cash only
Cash-only
Checks
Credit cards
Debit cards
NFC mobile payments


Now that the payment options are know it is possible to get only those restaurants having
electronic payment options among the possible ones:

In [3]:
# My coordinates (latitude and longitude)
my_latitude = 21.924238
my_longitude = -159.510035

radius = 1500  # Distance in meters

# Define electronic payment options
electronic_payments = ["Credit cards", 
                       "Debit cards", 
                       "NFC mobile payments"]

# Query to find restaurants near 
# the location with electronic payments
restaurant_close_numrev = meta.aggregate([
    {
        "$geoNear": {
            "near": {"type": "Point", "coordinates": [
                my_longitude, my_latitude]},
            # Field for distance calculation  
            "distanceField": "distance",
            # Max distance in meters  
            "maxDistance": radius,
            # Use spherical geometry for accurate distances  
            "spherical": True  
        }
    },
    {
        "$match": {
            # Check if any payment method matches
            "MISC.Payments": {"$in": electronic_payments}  
        }
    },
    {   # Sort by number of reviews (descending)
        "$sort": {"num_of_reviews": -1}  
    },
    {
        "$project": {"_id":1,
                     "name":1,
                     "num_of_reviews":1,
                     "avg_rating":1,
                     "MISC.Payments":1
                     }
    }
])

# Convert to list and print
results = list(restaurant_close_numrev)
for business in results:
    print(business)


{'_id': ObjectId('67c751b08c4ae1c3ac1c1eb5'), 'name': 'Kalaheo Cafe & Coffee Company', 'avg_rating': 4.7, 'num_of_reviews': 867, 'MISC': {'Payments': ['NFC mobile payments', 'Credit cards']}}
{'_id': ObjectId('67c751ad8c4ae1c3ac1be454'), 'name': 'Kauai Poke Co.Food Truck', 'avg_rating': 4.8, 'num_of_reviews': 58, 'MISC': {'Payments': ['Debit cards']}}
